In [ ]:
!pip install fastapi

In [2]:
import pandas as pd
import numpy as np
from pypdf import PdfReader
from Resources import DEGREES_IMPORTANCE
from spacy.lang.en import English
from pydantic import BaseModel, Field
from bson import ObjectId
import pandas as pd
import json
import ast
from fastapi import FastAPI
from fastapi.encoders import jsonable_encoder

In [7]:
reader = PdfReader("resumes/CV-ENGLISH_2.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()

In [8]:
text

"VICTOR BJORSVIK  \n \nLisbon, Portugal  | +47 917  83 150 | victorbjorsvik@hotmail.com  \n \nQUALIFICATION S                                                                                                                                                                        . \n• Leadership experience through serving as a Leading Private in the Royal Norwegian Navy and leading teams at Vender AS  \n• Creative and perceptive – influenced by international travels and studying abroad in Portugal, USA and Hungary  \n• 6 years of work experience in sales and service, and recent work experience in more quantitative subjects such as \nprogramming  for 6 months  and basic accounting  for 2 years  \n• Analytical skills through quantitative university courses ranging from linear algebra and calculus to big data analysis  \n• Strong communication and teamwork skills acquired through engaging with customers, clients, co -workers and team -mates  \n \nEDUCATION   \n \nNOVA SCHOOL OF BUSINESS AND 

In [9]:
dic = {"raw": [text]}
df2 = pd.DataFrame(dic)

In [10]:
class JobInfoExtraction:

    def __init__(self, skills_patterns_path, majors_patterns_path, degrees_patterns_path, jobs):
        # self.jobs = jobs[['Qualifications']]
        self.skills_patterns_path = skills_patterns_path
        self.majors_patterns_path = majors_patterns_path
        self.degrees_patterns_path = degrees_patterns_path
        self.degrees_importance = DEGREES_IMPORTANCE

    @staticmethod
    def match_majors_by_spacy(self, job):
        nlp = English()
        # Add the pattern to the matcher
        patterns_path = self.majors_patterns_path
        ruler = nlp.add_pipe("entity_ruler")
        ruler.from_disk(patterns_path)
        # Process some text
        doc1 = nlp(job)
        acceptable_majors = []
        for ent in doc1.ents:
            labels_parts = ent.label_.split('|')
            if labels_parts[0] == 'MAJOR':
                if labels_parts[2].replace('-', ' ') not in acceptable_majors:
                    acceptable_majors.append(labels_parts[2].replace('-', ' '))
                if labels_parts[2].replace('-', ' ') not in acceptable_majors:
                    acceptable_majors.append(labels_parts[2].replace('-', ' '))
        return acceptable_majors

    @staticmethod
    def match_degrees_by_spacy(self, job):
        nlp = English()
        # Add the pattern to the matcher
        patterns_path = self.degrees_patterns_path
        ruler = nlp.add_pipe("entity_ruler")
        ruler.from_disk(patterns_path)
        # Process some text
        doc1 = nlp(job)
        degree_levels = []
        for ent in doc1.ents:
            labels_parts = ent.label_.split('|')
            if labels_parts[0] == 'DEGREE':
                print((ent.text, ent.label_))
                if labels_parts[1] not in degree_levels:
                    degree_levels.append(labels_parts[1])
        return degree_levels

    @staticmethod
    def match_skills_by_spacy(self, job):
        nlp = English()
        patterns_path = self.skills_patterns_path
        ruler = nlp.add_pipe("entity_ruler")
        ruler.from_disk(patterns_path)
        # Process some text
        doc1 = nlp(job)
        job_skills = []
        for ent in doc1.ents:
            labels_parts = ent.label_.split('|')
            if labels_parts[0] == 'SKILL':
                print((ent.text, ent.label_))
                if labels_parts[1].replace('-', ' ') not in job_skills:
                    job_skills.append(labels_parts[1].replace('-', ' '))
        return job_skills

    @staticmethod
    def get_minimum_degree(self, degrees):
        """get the minimum degree that the candidate has"""
        d = {degree: self.degrees_importance[degree] for degree in degrees}
        return min(d, key=d.get)

    def extract_entities(self, jobs):
        # recognize and extract entities
        jobs['Minimum degree level'] = ""
        jobs['Acceptable majors'] = ""
        jobs['Skills'] = ""
        for i, row in jobs.iterrows():
            job = jobs['raw'][i].replace('. ', ' ')
            degrees = self.match_degrees_by_spacy(self, job)
            if len(degrees) != 0:
                jobs['Minimum degree level'][i] = self.get_minimum_degree(self, degrees)
            else:
                jobs['Minimum degree level'][i] = ""
            jobs['Acceptable majors'][i] = self.match_majors_by_spacy(self, job)
            jobs['Skills'][i] = self.match_skills_by_spacy(self, job)
        return jobs


In [11]:
class PyObjectId(ObjectId):
    @classmethod
    def __get_validators__(cls):
        yield cls.validate

    @classmethod
    def validate(cls, v):
        if not ObjectId.is_valid(v):
            raise ValueError("Invalid objectid")
        return ObjectId(v)

    @classmethod
    def __get_pydantic_json_schema__(cls, schema, _field):
        schema.update(type="string")
        return schema

class JobExtractedModel(BaseModel):
    id: PyObjectId = Field(default_factory=PyObjectId, alias="_id")
    minimum_degree_level: str = Field(...)
    acceptable_majors: list = Field(...)
    skills: list = Field(...)

    class Config:
        populate_by_name = True
        arbitrary_types_allowed = True
        json_encoders = {ObjectId: str}


In [12]:
def transform_dataframe_to_json(dataframe):

    # transforms the dataframe into json
    result = dataframe.to_json(orient="records")
    parsed = json.loads(result)
    json_data = json.dumps(parsed, indent=4)

    return json_data


def extraction(resume):
    degrees_patterns_path = 'Resources/data/degrees.jsonl'
    majors_patterns_path = 'Resources/data/majors.jsonl'
    skills_patterns_path = 'Resources/data/skills.jsonl'
    jobs = resume
    job_extraction = JobInfoExtraction(skills_patterns_path, majors_patterns_path, degrees_patterns_path, jobs)
    jobs = job_extraction.extract_entities(jobs)
    for i, row in jobs.iterrows():
        minimum_degree_level = jobs['Minimum degree level'][i]
        acceptable_majors = jobs['Acceptable majors'][i]
        skills = jobs['Skills'][i]

        job_extracted = JobExtractedModel(minimum_degree_level=minimum_degree_level if minimum_degree_level else '',
                                          acceptable_majors=acceptable_majors if acceptable_majors else [],
                                          skills=skills if skills else [])
        job_extracted = jsonable_encoder(job_extracted)
        # new_job_extracted = database.get_collection("jobsextracted").insert_one(job_extracted)
    jobs_json = transform_dataframe_to_json(jobs)
    return jobs_json


res = extraction(df2)

('university', 'DEGREE|BS-LEVEL')
('Master', 'DEGREE|MS-LEVEL')
('Bachelor', 'DEGREE|BS-LEVEL')


C:\Users\victo\AppData\Local\Temp\ipykernel_56756\4236327512.py:79: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  jobs['Minimum degree level'][i] = self.get_minimum_degree(self, degrees)
C:\Users\victo\AppData\Local\Temp\ipykernel_56756\4236

('accounting', 'SKILL|accounting')
('big data', 'SKILL|big-data')
('communication', 'SKILL|communications')
('BUSINESS', 'SKILL|business')
('Business', 'SKILL|business')
('Analytics', 'SKILL|analytics')
('Business', 'SKILL|business')
('analytics', 'SKILL|analytics')
('finance', 'SKILL|finance')
('data science', 'SKILL|data-science')
('analytics', 'SKILL|analytics')
('modelling', 'SKILL|modelling')
('Business Administration', 'SKILL|business-administration')
('Python', 'SKILL|python')
('logistic regression', 'SKILL|logistic-regression')
('models', 'SKILL|modelling')
('Web Development', 'SKILL|web-development')
('accounting', 'SKILL|accounting')
('Flask', 'SKILL|flask')
('HTML', 'SKILL|html')
('CSS', 'SKILL|css')
('Javascript', 'SKILL|javascript')
('testing', 'SKILL|testing')
('training', 'SKILL|training-model')
('Python', 'SKILL|python')
('Pandas', 'SKILL|pandas')
('SQL', 'SKILL|sql')
('Apache Spark', 'SKILL|apache-spark')
('Machine learning', 'SKILL|machine-learning')
('Tensorflow', 'S

C:\Users\victo\AppData\Local\Temp\ipykernel_56756\4236327512.py:83: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  jobs['Skills'][i] = self.match_skills_by_spacy(self, job)


In [13]:
df2.iloc[0].Skills

['accounting',
 'big data',
 'communications',
 'business',
 'analytics',
 'finance',
 'data science',
 'modelling',
 'business administration',
 'python',
 'logistic regression',
 'web development',
 'flask',
 'html',
 'css',
 'javascript',
 'testing',
 'training model',
 'pandas',
 'sql',
 'apache spark',
 'machine learning',
 'tensorflow',
 'keras',
 'c',
 'r',
 'tableau']